**Luca Procaccio**

**Politecnico di Milano**

**Codice persona: 10808417** 

**E-mail: luca.procaccio@mail.polimi.it**

# Knuth Morris Pratt algorithm

## Introduction

In string computation, the exact pattern matching problem is the problem of finding all the occurences of a pattern (string) P, in a text (string) S, where usually P is much shorter than S. 

For example the pattern could be the world “stella” and the text the whole Divina Commedia, or P can be the CCATTGTG motif and the text the human genome. 

One strategy to speed up the computation is to create an index on the pattern P and use this index to scan the text S in a more efficient way.

The Knuth-Morris-Pratt algorithm uses this approach. It searches for occurrences of a "word" W within a main "text string" S by employing the observation that when a mismatch occurs, the word itself embodies sufficient information to determine where the next match could begin, thus bypassing re-examination of previously matched characters.

## Kmp table

The goal of the table is to allow the algorithm not to match any character of text more than once. This function takes the pattern as an input and returns a matching table in the form of an array that contains, for each position, the lengths of longest proper prefix that is also a suffix. Hence, pattern is used to give the count of characters that can be skipped while matching with the main text.

In [1]:
import numpy as np

In [2]:
def kmp_table(pattern):
    
    # This if statement is partially redundant with the first one in kmp_search function.
    # Anyway, I've decided to keep it in case one want to use only the kmp_table function.
    
    if isinstance(pattern, str) == False:
        raise TypeError("the object's type must be: string")
        return
    
    # The table to be filled is created. It's an array of integers, 1 position longer than the pattern.
    table = np.empty((len(pattern) + 1), dtype = np.int32)
    
    pos = 1  # The current position we are computing in table.
    cnd = 0  # The zero-based index in the next character of the current candidate substring.
             
    table[0] = -1 # Initialization of the first element.
    
    while pos < len(pattern):
        if pattern[pos] == pattern[cnd]:
            table[pos] = table[cnd]
        else:
            table[pos] = cnd
            while cnd >= 0 and pattern[pos] != pattern[cnd]:
                cnd = table[cnd]
        pos = pos + 1
        cnd = cnd + 1
    
    table[len(pattern)] = cnd
    
    return table      

**Kmp_table complexity**

The time (and space) complexity of the table algorithm is O(k), where k is the length of pattern.

* The outer loop: pos is initialized to 1, the loop condition is pos < k, and pos is increased by 1 in every iteration of the loop. Thus the loop will take k - 1 iterations.

* The inner loop: cnd is initialized to 0 and gets increased by at most 1 in each outer loop iteration. T[cnd] is always less than cnd, so cnd gets decreased by at least 1 in each inner loop iteration; the inner loop condition is cnd ≥ 0. This means that the inner loop can execute at most as many times in total, as the outer loop has executed – each decrease of cnd by 1 in the inner loop needs to have a corresponding increase by 1 in the outer loop. Since the outer loop takes k - 1 iterations, the inner loop can take no more than k - 1 iterations in total.

Combined, the outer and inner loops take at most 2k - k  iterations. This corresponds to O(k) time complexity.

## Kmp search

At any given time, the algorithm is in a state determined by two integers:

* m, denoting the position within S(text) where the prospective match for P(pattern) begins,
* i, denoting the index of the currently considered character in P.

In each step the algorithm compares S\[ m+i \] with W\[ i \] and increments i if they are equal. If there is a mismatch, although the next possible match should begin at index m + i - T\[ i \], it is not needed actually to check any of the T\[ i \] characters after that, due to the previous iteration, so we continue searching from W\[ T [ i \] \] and S\[ i \].

Here I define the kmp search algorithm.

In [3]:
def kmp_search(text, pattern):
    
    # Raise an error if one of the two inputs it's not a string.
    if isinstance(text,str) == False or isinstance(pattern,str) == False:
        raise TypeError("Only strings!")
        return
    
    # Raise an error if the pattern is longer than text.
    if len(pattern) > len(text):
        raise TypeError("Pattern longer than text")
        return
    
    # Return an empty list in case of empty pattern.
    if not pattern:
        return []
    
    P = [] # List of positions in text at which pattern is found.
    nP = 0 # Number of positions in text at which pattern is found.
    
    j = 0 # Position of the current character in text.
    k = 0 # Position of the current character in pattern.
    
    table = kmp_table(pattern) # Partial match table.
    
    while j < len(text):
        if pattern[k] == text[j]:
            j = j + 1
            k = k + 1
            if k == len(pattern):
                P.append(j-k) 
                nP = nP + 1
                k = table[k]
        else:
            k = table[k]
            if k < 0:
                j = j + 1
                k = k + 1
    
    return P, nP                              

An example of the algorithm working.

In [4]:
pattern = 'ATC'
text = 'ATAGGCTATCATCCCTATC'

print('Text : "{}"\nPattern: "{}"\nFound matches at positions : {}'
      .format(text,pattern,kmp_search(text,pattern)))

Text : "ATAGGCTATCATCCCTATC"
Pattern: "ATC"
Found matches at positions : ([7, 10, 16], 3)


**Kmp_search complexity**

Assuming the prior existence of the table T, the search portion of the Knuth–Morris–Pratt algorithm has complexity O(n), where n is the length of S(text). All the computations are performed in the while loop and since at each iteration it executes one of the two branches, the loop can execute at most 2n times.

The first branch invariably increases i and does not change m, so that the index m + i of the currently scrutinized character of S is increased. The second branch adds i - T\[i \] to m this is always a positive number because the next possible match must occur at a higher index than m, so that index\[ i \] < i. Thus the location m of the beginning of the current potential match is increased. At the same time, the second branch leaves m + i unchanged, for m gets i - T\[ i \] added to it, and immediately after T\[ i \] gets assigned as the new value of i. 

Now, the loop ends if m + i = n; therefore, each branch of the loop can be reached at most n times, since they respectively increase either m + i or m, and m ≤ m + i: if m = n, then certainly m + i ≥ n, so that since it increases by unit increments at most, we must have had m + i = n at some point in the past, and therefore either way we would be done.

The time complexity is O(n).

**KMP algorithm total complexity**

Since the two portions of the algorithm have, respectively, complexities of O(k) and O(n), the complexity of the overall algorithm is O(n + k).

# Unit testing

Unit Testing is a software testing technique by means of which individual units of software i.e. group of computer program modules, usage procedures and operating procedures are tested to determine whether they are suitable for use or not. It is correlated with functional correctness of the independent modules.

In [5]:
import unittest

**Tests for kmp_table function**

In [9]:
class TestKmpTable(unittest.TestCase):
    
    # Error if pattern is not a string.
    def test_PnotString(self):
        self.assertRaises(TypeError, kmp_table, 123)
        
    # Standard.
    def test_standard(self):
        self.assertTrue((kmp_table('ACGACG') == np.array([-1, 0, 0, -1, 0, 0, 3], dtype = np.int32)).all())
        
    # Empty pattern as input.
    def test_emptyPattern(self):
        self.assertTrue((kmp_table('') == np.array([0], dtype = np.int32)).all())

**Tests for kmp_search function**

In [10]:
class TestKmpSearch(unittest.TestCase):
    
    # Error if pattern is not a string.
    def test_PnotString(self):
        pattern = 12
        text = 'ACGTGATCGATG'
        self.assertRaises(TypeError, kmp_search, (text, pattern))
    
    # Error if text is not a string.
    def test_TnotString(self):
        pattern = 'ATT'
        text = 123
        self.assertRaises(TypeError, kmp_search, (text,pattern))
    
    # Standard.
    def test_standard(self):
        pattern = 'AAA'
        text = 'AAA AAACGAAAG'
        self.assertEqual(kmp_search(text, pattern), ([0, 4, 9], 3))
        
    # No matches.
    def test_zeroMatches(self):
        pattern = 'AAA'
        text = 'GGTGTGTG'
        self.assertEqual(kmp_search(text,pattern), ([], 0))
        
    # Empty pattern as input.
    def test_emptyPattern(self):
        pattern = ''
        text = 'GGGACGAGCGGG'
        self.assertEqual(kmp_search(text, pattern), [])
    
    # Empty text as input.
    def test_emptyText(self):
        pattern = 'CCG'
        text = ''
        self.assertRaises(TypeError, kmp_search, (text,pattern))
        
    # Pattern longer than text.
    def test_longerPattern(self):
        pattern = 'ACGTGATCGATCGAG'
        text = 'ACGGAG'
        self.assertRaises(TypeError, kmp_search, (text, pattern))

We have to put some specifications in unittest.main(), otherwise the output would give an error.
The reason is that unittest.main looks at sys.argv and first parameter is what started Jupyter, therefore the error about kernel connection file not being a valid attribute. Passing explicit list to unittest.main will prevent Jupyter to look at sys.argv. Passing exit=False will prevent unittest.main to shutdown the kernel process.

In [11]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.006s

OK


# References

### KMP algorithm

https://en.wikipedia.org/wiki/Knuth–Morris–Pratt_algorithm

https://www.codesdope.com/blog/article/kmp-algorithm/


### Unit testing

https://medium.com/@vladbezden/using-python-unittest-in-ipython-or-jupyter-732448724e31

https://www.geeksforgeeks.org/unit-testing-software-testing/

https://docs.python.org/3/library/unittest.html